# Score:

- Thank you to Kaggle user [Serigine](https://www.kaggle.com/serigne) and his excellent [tutorial](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard) for helping me throughout this project

### Imports

In [22]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

import warnings
warnings.filterwarnings('ignore')

### Data

In [23]:
train = pd.read_csv('../../data/house_prices/train.csv')
test = pd.read_csv('../../data/house_prices/test.csv')

train_id = train['Id']
test_id = test['Id']

train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

### Initial preprocessing and merging data

Removing outliers were determined by an excerpt from the dataset auther [(link here)](https://ww2.amstat.org/publications/jse/v19n3/decock.pdf)

    " Although all known errors were corrected in the data, no observations have been removed due to unusual values  and all final residential sales from the initial data set are included in the data presented with this article. There are five observations that an instructor may wish to remove from the data set before giving it to students (a plot of SALE PRICE versus GR LIV AREA will quickly indicate these points). Three of them are true outliers (Partial Sales that likely don’t represent actual market values) and two of them are simply unusual sales (very large houses priced relatively appropriately). I would recommend removing any houses with more than 4000 square feet from the data set (which eliminates these five unusual observations) before assigning it to students. "

In [24]:
# remove outliers (see analysis notebook)
train = train.drop(train[(train['GrLivArea']>4000)].index)

# applies log(1+x) to all elements of the column
train["SalePrice"] = np.log1p(train["SalePrice"])

train_N = train.shape[0]
train_y = train.SalePrice.values

full = pd.concat((train, test)).reset_index(drop=True)
full.drop(['SalePrice'], axis=1, inplace=True)

### Fill empty values

In [25]:
# Fill empty values with None
for col in ('MiscFeature', 'PoolQC', 'Alley', 'Fence', 'FireplaceQu', 'GarageType', 'GarageFinish', 
            'GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
           'MasVnrType', 'MSSubClass'):
    full[col] = full[col].fillna("None")

# Fill empty values with 0
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
           'GarageYrBlt', 'GarageArea', 'GarageCars', 'MasVnrArea'):
    full[col] = full[col].fillna(0)
    
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
full["LotFrontage"] = full.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

for col in ('MSZoning', 'Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType', "Functional"):
    full[col] = full[col].fillna(full[col].mode()[0])

### Drop columns

In [26]:
full = full.drop(['Utilities', "MasVnrArea", "PoolQC", 'MiscFeature', 'MiscVal'], axis=1)

### Transforming numerical columns into categorical columns

In [27]:
for col in ('MSSubClass', 'OverallCond', 'YrSold', 'MoSold'):
    full[col] = full[col].apply(str)

### Label encodes all the categorical columns

In [28]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

for col in categorical_cols:
    lbl = LabelEncoder() 
    lbl.fit(list(full[col].values)) 
    full[col] = lbl.transform(list(full[col].values))

### Create new column Total Square Feet

In [29]:
full['TotalSF'] = full['TotalBsmtSF'] + full['1stFlrSF'] + full['2ndFlrSF']

### Removing skew

All of this skew analysis is thanks to Serginne

In [30]:
from scipy.special import boxcox1p
from scipy import stats
from scipy.stats import norm, skew

numeric_feats = full.dtypes[full.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = full[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness = skewness[abs(skewness) > 0.75]

skewed_features = skewness.index
lam = 0.15

for feat in skewed_features:
    full[feat] = boxcox1p(full[feat], lam)

### Turn categorical columns into dummy variables

In [31]:
full = pd.get_dummies(full)

### Split the merged data back into train and test data

In [32]:
train = full[:train_N]
test = full[train_N:]

### Machine Learning imports

In [33]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

### Validation scores

In [34]:
kf = KFold(5, shuffle=True, random_state=42).get_n_splits(train.values)

def rmsle_cv(model):
    rmse= np.sqrt(-cross_val_score(model, train.values, train_y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

### Creating different models

The parameters for each of the models were suggested by Serginne for being very robust

In [35]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

### Class to help stack different models

code here from Serginne

In [36]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

### Stacking different models

In [37]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

Stacking Averaged models score: 0.1077 (0.0076)


### Stacked Prediction

In [38]:
# Fitting
stacked_averaged_models.fit(train.values, train_y)

# Predicting train values
stacked_train_pred = stacked_averaged_models.predict(train.values)

# Predicting test values
stacked_pred = stacked_averaged_models.predict(test.values)
stacked_pred = np.expm1(stacked_pred) # exp(x) + 1 (reverses log1p from earlier)

print(rmsle(train_y, stacked_train_pred))

0.0771978043201


### XG Boost Prediction

In [39]:
xgb_params = {"colsample_bytree":0.4603, "gamma":0.0468, 
              "learning_rate":0.05, "max_depth":3, 
              "min_child_weight":1.7817, "n_estimators":2200,
              "reg_alpha":0.4640, "reg_lambda":0.8571,
              "subsample":0.5213, "silent":1,
              "random_state":7, "nthread":-1}

dtrain = xgb.DMatrix(train, train_y)
watchlist = [(dtrain, 'train')]

model = xgb.train(xgb_params, dtrain, 5000, watchlist, maximize=True, verbose_eval=1)
xgb_train_pred = model.predict(dtrain)

dtest = xgb.DMatrix(test)
xgb_pred = model.predict(dtest)
xgb_pred = np.expm1(xgb_pred)

print(rmsle(train_y, xgb_train_pred))

0.0782833508706


### Light Gradient Boost Prediction

In [40]:
# Fitting
model_lgb.fit(train, train_y)

# Predicting train values
lgb_train_pred = model_lgb.predict(train)

# Predicting test values
lgb_pred = model_lgb.predict(test.values)
lgb_pred = np.expm1(lgb_pred) # exp(x) + 1 (reverses log1p from earlier)

print(rmsle(train_y, lgb_train_pred))

0.0719611422303


### Merging all the different models

In [41]:
predictions = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

### Saving predictions to CSV

In [59]:
predictions = pd.DataFrame({
    'Id': test_id,
    'SalePrice': predictions
})

predictions.to_csv('../../submissions/more_xgb_5000_cycles.csv', index=False)